<a href="https://colab.research.google.com/github/nav13n/eva/blob/master/s4/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, bias=False),   #Input: 28x28 | Output: 26x26 | RF: 3
            nn.ReLU(),                         #Input: 28x28 | Output: 26x26 | RF: 3
            nn.BatchNorm2d(16),                #Input: 28x28 | Output: 26x26 | RF: 3
            nn.Dropout(p=0.1)                  #Input: 28x28 | Output: 26x26 | RF: 3
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, bias=False),  #Input: 26x26 | Output: 24x24 | RF: 5
            nn.ReLU(),                         #Input: 26x26 | Output: 24x24 | RF: 5
            nn.BatchNorm2d(32),                #Input: 26x26 | Output: 24x24 | RF: 5
            nn.Dropout(p=0.1)                  #Input: 26x26 | Output: 24x24 | RF: 5
        )

        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(32, 10, 1, bias=False),  #Input: 24x24 | Output: 24x24 | RF: 5
            nn.ReLU(),                         #Input: 24x24 | Output: 24x24 | RF: 5
            nn.BatchNorm2d(10),                #Input: 24x24 | Output: 24x24 | RF: 5
            nn.Dropout(p=0.1)                  #Input: 24x24 | Output: 24x24 | RF: 5
        )

        self.pool1 = nn.MaxPool2d(2, 2)        #Input: 24x24 | Output: 12x12 | RF: 5

        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(10, 16, 3, bias=False),  #Input: 12x12 | Output: 10x10 | RF: 7
            nn.ReLU(),                         #Input: 12x12 | Output: 10x10 | RF: 7
            nn.BatchNorm2d(16),                #Input: 12x12 | Output: 10x10 | RF: 7
            nn.Dropout(p=0.1)                  #Input: 12x12 | Output: 10x10 | RF: 7
        )

        self.conv_block_5 = nn.Sequential(
            nn.Conv2d(16, 16, 3, bias=False),  #Input: 10x10 | Output: 8x8   | RF: 9
            nn.ReLU(),                         #Input: 10x10 | Output: 8x8   | RF: 9
            nn.BatchNorm2d(16),                #Input: 10x10 | Output: 8x8   | RF: 9
            nn.Dropout(p=0.1)                  #Input: 10x10 | Output: 8x8   | RF: 9
        )

        self.conv_block_7 = nn.Sequential(
            nn.Conv2d(16, 16, 3, bias=False),  #Input: 8x8   | Output: 6x6   | RF: 11
            nn.ReLU(),                         #Input: 8x8   | Output: 6x6   | RF: 11
            nn.BatchNorm2d(16),                #Input: 8x8   | Output: 6x6   | RF: 11
            nn.Dropout(p=0.1)                  #Input: 8x8   | Output: 6x6   | RF: 11
        )

        self.conv_block_8 = nn.Sequential(
            nn.Conv2d(16, 10, 3, bias=False),  #Input: 6x6   | Output: 4x4   | RF: 13
            nn.ReLU(),                         #Input: 6x6   | Output: 4x4   | RF: 13
            nn.BatchNorm2d(10),                #Input: 6x6   | Output: 4x4   | RF: 13
            nn.Dropout(p=0.1)                  #Input: 6x6   | Output: 4x4   | RF: 13
        )

        self.average_pool = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.pool1(self.conv_block_3(self.conv_block_2(self.conv_block_1(x))))
        x = self.conv_block_8(self.conv_block_7(self.conv_block_5(self.conv_block_4(x))))
        x = self.average_pool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)





In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             320
             ReLU-10           [-1, 10, 24, 24]               0
      BatchNorm2d-11           [-1, 10, 24, 24]              20
          Dropout-12           [-1, 10, 24, 24]               0
        MaxPool2d-13           [-1, 10, 12, 12]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0703, Accuracy: 9831/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0465, Accuracy: 9889/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0200, Accuracy: 9941/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0218, Accuracy: 9927/10000 (99%)



loss=0.012260635383427143 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.83it/s]



Test set: Average loss: 0.0218, Accuracy: 9932/10000 (99%)

